In [1]:
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from googleapiclient.discovery import build
import os.path
import pickle
import re

class MarkdownToGoogleDoc:
    """
    Converts markdown meeting notes to Google Docs format with proper styling
    and formatting preservation.
    """

    SCOPES = ['https://www.googleapis.com/auth/documents']

    def __init__(self):
        self.creds = None
        self.service = None

    def read_markdown_file(self, file_path):
        """
        Read markdown content from a file.

        Args:
            file_path (str): Path to the markdown file

        Returns:
            str: Content of the markdown file

        Raises:
            FileNotFoundError: If the markdown file doesn't exist
            IOError: If there's an error reading the file
        """
        try:
            with open(file_path, 'r', encoding='utf-8') as file:
                return file.read()
        except FileNotFoundError:
            raise FileNotFoundError(f"Markdown file not found: {file_path}")
        except IOError as e:
            raise IOError(f"Error reading markdown file: {str(e)}")

    def authenticate(self):
        """Handle Google Docs API authentication in Colab environment."""
        if os.path.exists('token.pickle'):
            with open('token.pickle', 'rb') as token:
                self.creds = pickle.load(token)

        if not self.creds or not self.creds.valid:
            if self.creds and self.creds.expired and self.creds.refresh_token:
                self.creds.refresh(Request())
            else:
                flow = InstalledAppFlow.from_client_secrets_file(
                    'credentials.json', self.SCOPES)
                self.creds = flow.run_local_server(port=0)

            with open('token.pickle', 'wb') as token:
                pickle.dump(self.creds, token)

        self.service = build('docs', 'v1', credentials=self.creds)

    def create_document(self, title):
        """Create a new Google Doc with the specified title."""
        try:
            document = self.service.documents().create(body={'title': title}).execute()
            return document.get('documentId')
        except Exception as e:
            raise Exception(f"Error creating document: {str(e)}")

    def parse_markdown(self, markdown_text):
        """
        Parse markdown text and convert it to Google Docs API requests.
        Returns a list of requests for the Google Docs API.
        """
        requests = []
        current_index = 1  # Start after title
        lines = markdown_text.split('\n')

        # Process title (H1)
        title_match = re.match(r'^#\s+(.+)', lines[0])
        if title_match:
            requests.append({
                'updateParagraphStyle': {
                    'range': {'startIndex': 0, 'endIndex': len(lines[0])},
                    'paragraphStyle': {'namedStyleType': 'HEADING_1'},
                    'fields': 'namedStyleType'
                }
            })
            current_index += len(lines[0]) + 1

        for line in lines[1:]:
            # Handle H2 headers
            if line.startswith('## '):
                text = line[3:]
                requests.append({
                    'updateParagraphStyle': {
                        'range': {'startIndex': current_index,
                                 'endIndex': current_index + len(text)},
                        'paragraphStyle': {'namedStyleType': 'HEADING_2'},
                        'fields': 'namedStyleType'
                    }
                })

            # Handle H3 headers
            elif line.startswith('### '):
                text = line[4:]
                requests.append({
                    'updateParagraphStyle': {
                        'range': {'startIndex': current_index,
                                 'endIndex': current_index + len(text)},
                        'paragraphStyle': {'namedStyleType': 'HEADING_3'},
                        'fields': 'namedStyleType'
                    }
                })

            # Handle checkboxes and mentions
            elif re.match(r'-\s+\[\s?\]', line):
                requests.append({
                    'createParagraphBullets': {
                        'range': {'startIndex': current_index,
                                 'endIndex': current_index + len(line)},
                        'bulletPreset': 'CHECKBOX'
                    }
                })

                # Style mentions (@name)
                mentions = re.finditer(r'@(\w+)', line)
                for mention in mentions:
                    requests.append({
                        'updateTextStyle': {
                            'range': {
                                'startIndex': current_index + mention.start(),
                                'endIndex': current_index + mention.end()
                            },
                            'textStyle': {'bold': True, 'foregroundColor': {
                                'color': {'rgbColor': {'blue': 0.8, 'red': 0.2, 'green': 0.2}}
                            }},
                            'fields': 'bold,foregroundColor'
                        }
                    })

            # Handle bullet points
            elif line.startswith('* ') or line.startswith('- '):
                indent_level = (len(line) - len(line.lstrip())) // 2
                requests.append({
                    'createParagraphBullets': {
                        'range': {'startIndex': current_index,
                                 'endIndex': current_index + len(line)},
                        'bulletPreset': 'BULLET',
                        'indentLevel': indent_level
                    }
                })

            current_index += len(line) + 1

        return requests

    def update_document(self, doc_id, markdown_text):
        """Update the document with formatted content."""
        try:
            # Insert the content
            self.service.documents().batchUpdate(
                documentId=doc_id,
                body={'requests': self.parse_markdown(markdown_text)}
            ).execute()
        except Exception as e:
            raise Exception(f"Error updating document: {str(e)}")

def main():
    # File path for the markdown file
    markdown_file_path = 'meeting_notes.md'

    try:
        converter = MarkdownToGoogleDoc()

        # Read markdown content from file
        markdown_content = converter.read_markdown_file(markdown_file_path)

        # Extract title from first line
        title = re.match(r'^#\s+(.+)', markdown_content.split('\n')[0])
        doc_title = title.group(1) if title else "Meeting Notes"

        # Process the document
        converter.authenticate()
        doc_id = converter.create_document(doc_title)
        converter.update_document(doc_id, markdown_content)
        print(f"Document created successfully! ID: {doc_id}")

    except FileNotFoundError as e:
        print(f"File error: {str(e)}")
    except Exception as e:
        print(f"Error: {str(e)}")

if __name__ == "__main__":
    main()

Error: [Errno 2] No such file or directory: 'credentials.json'
